In [66]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from skimage import exposure

In [67]:
# Directory paths for the dataset
dataset_path = './Dataset/MyPedestrian'
pedestrian_path = os.path.join(dataset_path, 'Pedestrian')
non_pedestrian_path = os.path.join(dataset_path, 'NonPedestrian')


In [68]:
def extract_hog_features(image):
    # Resize ảnh về kích thước 128x256
    image_resized = cv2.resize(image, (256, 128))
    
    # Chuyển ảnh sang grayscale
    image_gray = cv2.cvtColor(image_resized, cv2.COLOR_BGR2GRAY)

    # Trích xuất đặc trưng HOG
    features, _ = hog(image_gray, 
                      orientations=9, 
                      pixels_per_cell=(8, 8), 
                      cells_per_block=(2, 2), 
                      block_norm='L2-Hys', 
                      feature_vector=True, 
                      visualize=True)
    
    return features

In [69]:
# Đọc ảnh từ thư mục và trả về đặc trưng HOG
def load_images_from_folder(folder, label):
    features_list = []
    labels = []
    for filename in os.listdir(folder):
        image_path = os.path.join(folder, filename)
        image = cv2.imread(image_path)
        
        if image is not None:
            features = extract_hog_features(image)
            features_list.append(features)
            labels.append(label)
    
    return features_list, labels

In [70]:
# Tải dữ liệu và trích xuất đặc trưng HOG
pedestrian_features, pedestrian_labels = load_images_from_folder(pedestrian_path, label=1)  # Label 1 cho người đi bộ
non_pedestrian_features, non_pedestrian_labels = load_images_from_folder(non_pedestrian_path, label=0)  # Label 0 cho không phải người đi bộ

# Kiểm tra kích thước các đặc trưng
print(f"Shape of first feature: {len(pedestrian_features[0])}")
print(f"Shape of second feature: {len(non_pedestrian_features[0])}")

Shape of first feature: 16740
Shape of second feature: 16740


In [77]:
# Đảm bảo tất cả các đặc trưng có cùng kích thước
X = np.array(pedestrian_features + non_pedestrian_features)
y = np.array(pedestrian_labels + non_pedestrian_labels)

# Kiểm tra kích thước của X sau khi kết hợp
print(f"Shape of combined features (X): {X.shape}")

# Chia dữ liệu thành tập huấn luyện và kiểm tra (tỉ lệ 70:30)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42,  stratify=y)

# Khởi tạo và huấn luyện mô hình KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

# Khởi tạo và huấn luyện mô hình LogisticRegression
log_reg = LogisticRegression()  # Đảm bảo mô hình có thể hội tụ
log_reg.fit(X_train, y_train)

# Dự đoán và in báo cáo phân loại cho KNeighborsClassifier
y_pred_knn = knn.predict(X_test)
print("KNeighborsClassifier - Classification Report:")
print(classification_report(y_test, y_pred_knn))

# Dự đoán và in báo cáo phân loại cho LogisticRegression
y_pred_log_reg = log_reg.predict(X_test)
print("LogisticRegression - Classification Report:")
print(classification_report(y_test, y_pred_log_reg))


Shape of combined features (X): (40, 16740)
KNeighborsClassifier - Classification Report:
              precision    recall  f1-score   support

           0       0.62      1.00      0.77         5
           1       1.00      0.57      0.73         7

    accuracy                           0.75        12
   macro avg       0.81      0.79      0.75        12
weighted avg       0.84      0.75      0.74        12

LogisticRegression - Classification Report:
              precision    recall  f1-score   support

           0       0.71      1.00      0.83         5
           1       1.00      0.71      0.83         7

    accuracy                           0.83        12
   macro avg       0.86      0.86      0.83        12
weighted avg       0.88      0.83      0.83        12



In [78]:
#Truong hop doc nguoi khong di bo truoc
X = np.array(non_pedestrian_features + pedestrian_features)
y = np.array(non_pedestrian_labels + pedestrian_labels)

# Kiểm tra kích thước của X sau khi kết hợp
print(f"Shape of combined features (X): {X.shape}")

# Chia dữ liệu thành tập huấn luyện và kiểm tra (tỉ lệ 70:30)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Khởi tạo và huấn luyện mô hình KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

# Khởi tạo và huấn luyện mô hình LogisticRegression
log_reg = LogisticRegression()  # Đảm bảo mô hình có thể hội tụ
log_reg.fit(X_train, y_train)

# Dự đoán và in báo cáo phân loại cho KNeighborsClassifier
y_pred_knn = knn.predict(X_test)
print("KNeighborsClassifier - Classification Report:")
print(classification_report(y_test, y_pred_knn))

# Dự đoán và in báo cáo phân loại cho LogisticRegression
y_pred_log_reg = log_reg.predict(X_test)
print("LogisticRegression - Classification Report:")
print(classification_report(y_test, y_pred_log_reg))


Shape of combined features (X): (40, 16740)
KNeighborsClassifier - Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.86      0.80         7
           1       0.75      0.60      0.67         5

    accuracy                           0.75        12
   macro avg       0.75      0.73      0.73        12
weighted avg       0.75      0.75      0.74        12

LogisticRegression - Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         7
           1       1.00      1.00      1.00         5

    accuracy                           1.00        12
   macro avg       1.00      1.00      1.00        12
weighted avg       1.00      1.00      1.00        12

